In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#hyper parameters
num_epochs = 4
batch_size = 3
learning_rate = 0.01

In [ ]:
#Dataset, CIFAR10 (pictures of animals)
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train = True,
                                          transform=transform, download=True)
                                          
test_dataset = torchvision.datasets.CIFAR10(root='./data', train = False,
                                          transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = batch_size,
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = batch_size,
                                           shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
examples = iter(train_loader)
samples, labels = examples.next()
print(labels.shape)
'''
for i in range(6):
    samples,_ = examples.next()
    for j in range(samples.shape[0]):
        plt.subplot(2, 2, j+1)
        plt.imshow(samples[j][0], cmap='gray')
    plt.show()'''

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120) #første argument skal være lig det forrige lags shape. Hvis den har shape [4, 32, 7, 9] skal første argument her være 32*7*9
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.pool(out)
        out = F.relu(self.conv2(out))
        out = self.pool(out)
        out = out.view(-1,16*5*5)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [ ]:
model = ConvNet().to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss() #for multiclass classification
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
type(train_loader)

In [8]:
#training loop
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        print(type(images))
        #origin shape: [4, 3, 32, 32] = 4, 3, 1024
        #plt.imshow(images, cmap='gray')
        #print("images = ", images)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        print("outputs shape: ", outputs.shape)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        """
        if (i+1) % 100 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
    """

outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor'>
outputs shape:  torch.Size([4, 10])
<class 'torch.Tensor

KeyboardInterrupt: 

In [ ]:
#test
with torch.no_grad(): 
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels, in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    acc = 100 * n_correct / n_samples
    print(f'accuracy of the network = {acc:.4f}')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')